In [88]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [89]:
passenger_2020=pd.read_csv('processed_data/Tridelta/tridelta_route_timeseries_2020.csv')  
passenger_2019=pd.read_csv('processed_data/Tridelta/tridelta_route_timeseries_2019.csv')
stops=pd.read_csv('processed_data/tridelta/tridelta_stops.csv')
routes=pd.read_csv('tridelta_routes.csv')
pd.to_datetime(passenger_2019['Date'], infer_datetime_format=True)  
pd.to_datetime(passenger_2020['Date'], infer_datetime_format=True)  
passenger_2020['inputDates'] = pd.to_datetime(passenger_2020['Date'])
passenger_2019['inputDates'] = pd.to_datetime(passenger_2019['Date'])
passenger_2020['dayOfWeek'] = passenger_2020['inputDates'].dt.day_name()
passenger_2019['dayOfWeek'] = passenger_2019['inputDates'].dt.day_name()
passenger_2020['Day']=np.where((passenger_2020['dayOfWeek']=='Saturday')|(passenger_2020['dayOfWeek']=='Sunday')|(passenger_2020['dayOfWeek']=='Friday'), 'weekend', 'weekday')
passenger_2019['Day']=np.where((passenger_2019['dayOfWeek']=='Saturday')|(passenger_2019['dayOfWeek']=='Sunday')|(passenger_2019['dayOfWeek']=='Friday'), 'weekend', 'weekday')

passenger_2020=passenger_2020[passenger_2020['dayOfWeek']!='Friday']
passenger_2019=passenger_2019[passenger_2019['dayOfWeek']!='Friday']

passenger_2020['Week_Number'] = passenger_2020['inputDates'].dt.strftime('%U')
passenger_2019['Week_Number'] = passenger_2019['inputDates'].dt.strftime('%U')


In [90]:
passenger_2020_weekday=passenger_2020[passenger_2020['Day']=='weekday']
passenger_2019_weekday=passenger_2019[passenger_2019['Day']=='weekday']

passenger_2020_weekend=passenger_2020[passenger_2020['Day']=='weekend']
passenger_2019_weekend=passenger_2019[passenger_2019['Day']=='weekend']

## Max activity 2019 & 2020

In [91]:
passenger_2020_weekday_grouped=passenger_2020_weekday.groupby(['Route']).mean().reset_index()
passenger_2019_weekday_grouped=passenger_2019_weekday.groupby(['Route']).mean().reset_index()

passenger_2020_weekday_grouped=passenger_2020_weekday_grouped[['Route','Passenger on']]
passenger_2019_weekday_grouped=passenger_2019_weekday_grouped[['Route','Passenger on']]

passenger_2020_weekend_grouped=passenger_2020_weekend.groupby(['Route','Day']).mean().reset_index()
passenger_2019_weekend_grouped=passenger_2019_weekend.groupby(['Route','Day']).mean().reset_index()

passenger_2020_weekend_grouped=passenger_2020_weekend_grouped[['Route','Passenger on']]
passenger_2019_weekend_grouped=passenger_2019_weekend_grouped[['Route','Passenger on']]

In [92]:
pass_on_week_2020=passenger_2020_weekday_grouped.sort_values(by='Passenger on',ascending=False)
routes_pass_on_max_weekday_2020=pass_on_week_2020.iloc[0:10, :]



pass_on_weekend_2020=passenger_2020_weekend_grouped.sort_values(by='Passenger on',ascending=False)
routes_pass_on_max_weekend_2020=pass_on_weekend_2020.iloc[0:10, :]




pass_on_week_2019=passenger_2019_weekday_grouped.sort_values(by='Passenger on',ascending=False)
routes_pass_on_max_weekday_2019=pass_on_week_2019.iloc[0:10, :]



pass_on_weekend_2019=passenger_2019_weekend_grouped.sort_values(by='Passenger on',ascending=False)
routes_pass_on_max_weekend_2019=pass_on_weekend_2019.iloc[0:10, :]


In [93]:
with pd.ExcelWriter('TRIDELTA_Top_10_Routes_by_avg_pass.xlsx') as writer1:
    routes_pass_on_max_weekday_2020.to_excel(writer1, sheet_name='Weekday 2020', index = False)
    routes_pass_on_max_weekday_2019.to_excel(writer1, sheet_name='Weekday 2019', index = False)
    routes_pass_on_max_weekend_2020.to_excel(writer1, sheet_name='Weekend 2020', index = False)
    routes_pass_on_max_weekend_2019.to_excel(writer1, sheet_name='Weekend 2019', index = False)
    

In [94]:
routes_stops_weekday_2020=routes[routes['Route'].isin(routes_pass_on_max_weekday_2020['Route'])]
routes_stops_weekday_2019=routes[routes['Route'].isin(routes_pass_on_max_weekday_2019['Route'])]
routes_stops_weekend_2020=routes[routes['Route'].isin(routes_pass_on_max_weekend_2020['Route'])]
routes_stops_weekday_2020=routes[routes['Route'].isin(routes_pass_on_max_weekday_2020['Route'])]

In [95]:
routes_stops_weekday_2020_merged_tract=routes_stops_weekday_2020.merge(stops,left_on='Stop ID', right_on='stop_id')[['Route', 'Stop ID', 'TRACTCE']]
routes_stops_weekday_2019_merged_tract=routes_stops_weekday_2019.merge(stops,left_on='Stop ID', right_on='stop_id')[['Route', 'Stop ID', 'TRACTCE']]

routes_stops_weekend_2020_merged_tract=routes_stops_weekend_2020.merge(stops,left_on='Stop ID', right_on='stop_id')[['Route', 'Stop ID', 'TRACTCE']]
routes_stops_weekend_2019_merged_tract=routes_stops_weekday_2019.merge(stops,left_on='Stop ID', right_on='stop_id')[['Route', 'Stop ID', 'TRACTCE']]

In [96]:
routes_stops_weekday_2020_merged_tract_grouped=routes_stops_weekday_2020_merged_tract.groupby(['Route'])['TRACTCE'].unique()
routes_stops_weekday_2019_merged_tract_grouped=routes_stops_weekday_2019_merged_tract.groupby(['Route'])['TRACTCE'].unique()

routes_stops_weekend_2020_merged_tract_grouped=routes_stops_weekend_2020_merged_tract.groupby(['Route'])['TRACTCE'].unique()
routes_stops_weekend_2019_merged_tract_grouped=routes_stops_weekend_2019_merged_tract.groupby(['Route'])['TRACTCE'].unique()

In [97]:
with pd.ExcelWriter('TRIDELTA_Top_10_Routes_tracts.xlsx') as writer1:
    routes_stops_weekday_2020_merged_tract_grouped.to_excel(writer1, sheet_name='Weekday 2020', index = True)
    routes_stops_weekday_2019_merged_tract_grouped.to_excel(writer1, sheet_name='Weekday 2019', index = True)
    routes_stops_weekend_2020_merged_tract_grouped.to_excel(writer1, sheet_name='Weekend 2020', index = True)
    routes_stops_weekend_2019_merged_tract_grouped.to_excel(writer1, sheet_name='Weekend 2019', index = True)